In [23]:
# Import Dependencies
import os
import pandas as pd
import requests
import json
from config import (aqs_api_key)
from config import (email_address)

# https://aqs.epa.gov/aqsweb/documents/data_api.html#signup for api information

list_url = "https://aqs.epa.gov/data/api/list/states"
params = {"email" : email_address, "key" : aqs_api_key}

In [24]:
response = requests.get(list_url, params = params)
print(response.url)

https://aqs.epa.gov/data/api/list/states?email=trent.e.little%40gmail.com&key=sandmallard17


In [25]:
state_list = response.json()
state_list

{'Header': [{'status': 'Success',
   'request_time': '2020-08-08T11:09:04-04:00',
   'url': 'https://aqs.epa.gov/data/api/list/states?email=trent.e.little%40gmail.com&key=sandmallard17',
   'rows': 56}],
 'Data': [{'code': '01', 'value_represented': 'Alabama'},
  {'code': '02', 'value_represented': 'Alaska'},
  {'code': '04', 'value_represented': 'Arizona'},
  {'code': '05', 'value_represented': 'Arkansas'},
  {'code': '06', 'value_represented': 'California'},
  {'code': '08', 'value_represented': 'Colorado'},
  {'code': '09', 'value_represented': 'Connecticut'},
  {'code': '10', 'value_represented': 'Delaware'},
  {'code': '11', 'value_represented': 'District Of Columbia'},
  {'code': '12', 'value_represented': 'Florida'},
  {'code': '13', 'value_represented': 'Georgia'},
  {'code': '15', 'value_represented': 'Hawaii'},
  {'code': '16', 'value_represented': 'Idaho'},
  {'code': '17', 'value_represented': 'Illinois'},
  {'code': '18', 'value_represented': 'Indiana'},
  {'code': '19', '